In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Load data from CSV
data = pd.read_csv("input.csv")

# Convert data to a numpy array
data_np = data.to_numpy()

# Normalize the data (you can use other normalization methods too)
data_np = (data_np - data_np.mean(axis=0)) / data_np.std(axis=0)

# Define the input sequence length and number of features
sequence_length = 10
num_features = data_np.shape[1]

# Prepare input data and labels for training
X = []
y = []
for i in range(len(data_np) - sequence_length):
    X.append(data_np[i:i + sequence_length])
    y.append(data_np[i + sequence_length])

X = np.array(X)
y = np.array(y)

# Convert numpy arrays to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Create a custom dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

# Create data loaders for training and validation
batch_size = 64
train_dataset = TimeSeriesDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(CNNFeatureExtractor, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(input_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * (sequence_length // 4), output_channels),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


In [ ]:
# Initialize the CNN model
input_channels = num_features
output_channels = 128  # Number of output features for feature extraction
model = CNNFeatureExtractor(input_channels, output_channels)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 50

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x.transpose(1, 2))
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}")

# Save the trained model
torch.save(model.state_dict(), "cnn_feature_extractor.pth")
